In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
%matplotlib inline

In [2]:
df = pd.read_csv("energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
df.shape

(19735, 29)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [9]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [15]:
linear_model.fit(df[['T2']],df[['T6']])
r2 = linear_model.score(df[['T2']],df[['T6']])
r2

0.6418990830855493

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [20]:
df.drop(columns = ['date','lights'], inplace = True)

In [21]:
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [22]:
features_df = normalised_df.drop(columns=['Appliances'])
target = normalised_df['Appliances']

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, target,test_size=0.3, random_state=42)

In [25]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [26]:
linear_model.fit(x_train, y_train)

LinearRegression()

In [27]:
predicted_values = linear_model.predict(x_test)

In [29]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values) 
round(mae, 3)

0.05

In [30]:
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

In [31]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

In [35]:
#coefficient
from sklearn.metrics import r2_score
rmse = r2_score(y_test, predicted_values)
round(rmse, 2)

0.15

In [39]:
linear_model.coef_.min()

-0.4566979483385004

In [44]:
pd.Series(linear_model.coef_, linear_model.feat_names_in_).sort_values()

AttributeError: 'LinearRegression' object has no attribute 'feat_names_in_'

In [45]:
from sklearn.linear_model import Ridge

In [51]:
ridge = Ridge(alpha = 0.4)
ridge.fit(x_train, y_train)
ridgepred = ridge.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, ridgepred))
rmse

0.08753385704628003

In [52]:
from sklearn.linear_model import Lasso

In [54]:
lasso = Ridge(alpha = 0.001)
lasso.fit(x_train, y_train)

Ridge(alpha=0.001)

In [65]:
las = lasso.coef_.nonzero()
len(las[0])

26

In [60]:
lassopred = ridge.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, lassopred))
rmse
round(rmse, 3)

0.088